# Импорт библиотек

In [78]:
import re

import pandas as pd
import nltk
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
import numpy as np

from tqdm import tqdm

nltk.download("punkt")

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
data = pd.read_csv("../data/all_data.csv")

In [53]:
data.describe()

,text,title
count,29899,29899
unique,29899,27052
top,протопоп аввакум написал житие по благословени...,классика зарубежного рассказа
freq,1,26


# Разделение данных

In [54]:
train_df, val_df = train_test_split(data, test_size=0.2, random_state=42)

In [55]:
len(train_df), len(val_df)

(23919, 5980)

In [56]:
val_df.describe()

,text,title
count,5980,5980
unique,5980,5807
top,френдзона 3. марта 3 ну и алекс! я не перестав...,наследник ссср - фрагмент
freq,1,7


Заметим, что в валидацию попали несколько текстов с одинаковыми названиями. Это не очень хорошо, так как может исказить результаты оценки модели.

Переделаем разделение.

In [57]:
def split_with_controlled_test_size(data, target_test_size=0.2, random_state=42):
    np.random.seed(random_state)

    title_groups = data.groupby('title').apply(lambda x: x.index.tolist()).to_dict()

    unique_titles = list(title_groups.keys())
    np.random.shuffle(unique_titles)

    train_indices = []
    test_indices = []

    target_test_count = int(len(data) * target_test_size)

    for title in unique_titles:
        indices = title_groups[title]

        if len(test_indices) < target_test_count:
            test_idx = np.random.choice(indices, 1)[0]
            test_indices.append(test_idx)
            train_indices.extend([idx for idx in indices if idx != test_idx])
        else:
            train_indices.extend(indices)

    return data.iloc[train_indices], data.iloc[test_indices]

train_df, val_df = split_with_controlled_test_size(data)

/var/folders/wl/_8h74t5s5hz2ntrny0kjx_pw0000gn/T/ipykernel_36720/2556919616.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  title_groups = data.groupby('title').apply(lambda x: x.index.tolist()).to_dict()


In [58]:
print(f"Общий размер данных: {len(data)}")
print(f"Тренировочная выборка: {len(train_df)} записей ({len(train_df)/len(data)*100:.1f}%)")
print(f"Валидационная выборка: {len(val_df)} записей ({len(val_df)/len(data)*100:.1f}%)")

Общий размер данных: 29899
Тренировочная выборка: 23920 записей (80.0%)
Валидационная выборка: 5979 записей (20.0%)


In [59]:
val_df.describe()

,text,title
count,5979,5979
unique,5979,5979
top,вторая мировая война. тихоокеанский театр воен...,нагие и мёртвые
freq,1,1


Теперь в валидации нет повторяющихся заголовков.

In [60]:
val_df.to_csv("../data/training_data/val_df.csv", index=False)

val_df.to_csv("../data/val_df.csv", index=False)# Аугментация данных

Аугментировать будем только train набор, чтобы не произошла утечка.

In [61]:
train_df.reset_index(drop=True, inplace=True)

In [62]:
def split_text_into_chunks(text, sentences_per_chunk=3):
    sentences = sent_tokenize(text, language="russian")
    chunks = []
    for i in range(0, len(sentences), sentences_per_chunk):
        chunk = " ".join(sentences[i:i+sentences_per_chunk])
        chunks.append(chunk)
    return chunks

In [63]:
augmented_rows = []
for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
    chunks = split_text_into_chunks(row.text, sentences_per_chunk=3)
    for chunk in chunks:
        augmented_rows.append({"text": chunk, "title": row.title})

100%|██████████| 23920/23920 [00:23<00:00, 1007.47it/s]


In [86]:
augmented_dataset = pd.DataFrame(augmented_rows)

In [87]:
augmented_dataset.describe()

,text,title
count,1295472,1295472
unique,1267145,21472
top,. . .,чистый хозяин собственного мира
freq,203,13972


## Почистим строки, в которых нет текста и удалим лидирующую пунктуацию

In [88]:
def is_meaningful(text):
    return bool(re.search(r"[А-Яа-яA-Za-z0-9]", text))

def clean_leading_punct(text):
    return re.sub(r"^[^\wА-Яа-я0-9]+", "", text).strip()

In [89]:
augmented_dataset = augmented_dataset[augmented_dataset.text.apply(is_meaningful)].reset_index(drop=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"[^\w\s,.!?-]", " ", regex=True)
augmented_dataset.text = augmented_dataset.text.str.replace(r"\s+", " ", regex=True).str.strip()
augmented_dataset.describe()

,text,title
count,1295138,1295138
unique,1267097,21472
top,если же кого-то заинтересует более подробно то...,чистый хозяин собственного мира
freq,50,13972


Удалим строки, с небольшим количеством данных

In [90]:
augmented_dataset = augmented_dataset[augmented_dataset.text.str.strip().str.len() > 10]

In [91]:
augmented_dataset.describe()

,text,title
count,1294786,1294786
unique,1266812,21467
top,"это не значит, что остальным жителям планеты н...",чистый хозяин собственного мира
freq,50,13972


Заметим, что у нас очень много текстов с одинаковыми названиями. Это может плохо повлиять на модель, если она будет видеть одни и те же названия. Оставим только по 500 каждого

In [92]:
max_per_title = 500
augmented_dataset = augmented_dataset.groupby("title").head(max_per_title).reset_index(drop=True)

print(augmented_dataset.title.value_counts().head(10))

title
на восток и обратно, если повезёт         500
перстень мастера                          500
если есть 20 минут                        500
умереть чудесным майским днем             500
прокляття василя яблуковича               500
тревожное очарование                      500
пророку ев ап иоанну богослову чудотв!    500
между мирами                              500
дорога цвета собаки                       500
квартирантка                              500
Name: count, dtype: int64


In [93]:
augmented_dataset.describe()

,text,title
count,836328,836328
unique,820297,21467
top,"надеюсь, что данные мысли волнуют не только ме...","на восток и обратно, если повезёт"
freq,50,500


In [94]:
augmented_dataset.to_csv("../data/training_data/train_df.csv", index=False)

## Посмотрим на распределение названий по длине после разделения

In [95]:
train_df = augmented_dataset.copy()

In [96]:
dup_titles = (
    augmented_dataset.groupby("text")["title"]
    .nunique()
    .reset_index()
    .query("title > 1")
)

print(f"Текстов с одинаковыми содержаниями, но разными названиями: {len(dup_titles)}")

Текстов с одинаковыми содержаниями, но разными названиями: 3458


In [97]:
data["title_len"] = data.title.apply(lambda x: len(str(x).split()))
length_counts = data.groupby('title_len').size()
train_df["title_len"] = train_df.title.apply(lambda x: len(str(x).split()))
length_counts_aug = train_df.groupby('title_len').size()
val_df["title_len"] = val_df.title.apply(lambda x: len(str(x).split()))
length_counts_aug_val = val_df.groupby('title_len').size()

/var/folders/wl/_8h74t5s5hz2ntrny0kjx_pw0000gn/T/ipykernel_36720/4038663461.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [98]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=length_counts.index,
    y=length_counts.values,
    name="Без аугментации",
    marker_color="blue"
))

fig.add_trace(go.Bar(
    x=length_counts_aug.index,
    y=length_counts_aug.values,
    name="train после аугментации",
    marker_color="orange"
))

fig.add_trace(go.Bar(
    x=length_counts_aug_val.index,
    y=length_counts_aug_val.values,
    name="test после аугментации",
    marker_color="green"
))

fig.update_layout(
    title="Сравнение распределения длин названий до и после аугментации",
    xaxis_title="Длина названия (слов)",
    yaxis_title="Количество названий",
    barmode="group",
    bargap=0.2,
    bargroupgap=0.1,
    width=1000,
    height=500
)

fig.show()